In [5]:
import random

class MemoryTier:
    def __init__(self, name, size, latency):
        self.name = name
        self.size = size
        self.latency = latency
        self.data = {}
        self.access_order = []

    def access(self, address):
        if address in self.data:
            if address in self.access_order:
                self.access_order.remove(address)
            self.access_order.append(address)
            return self.data[address], self.latency
        return None, self.latency

    def write(self, address, value):
        if len(self.data) >= self.size:
            evicted_address = self.access_order.pop(0)
            del self.data[evicted_address]
            print(f"Evicted address {evicted_address} from {self.name}")
        self.data[address] = value
        self.access_order.append(address)

class TieredMemorySystem:
    def __init__(self, dram_size, dram_latency, dram_bandwidth, nvm_size, nvm_latency, nvm_bandwidth):
        self.dram = MemoryTier("DRAM", dram_size, dram_latency)
        self.nvm = MemoryTier("NVM", nvm_size, nvm_latency)
        self.dram_bandwidth = dram_bandwidth
        self.nvm_bandwidth = nvm_bandwidth
        self.total_copy_time = 0
        self.total_migration_time = 0
        self.total_remap_time = 0
        self.migration_count = 0
        self.copy_count = 0

    def migrate_to_dram(self, address, value):
        try:
            # Simulate memory copying time
            data_size = 1024  # Assume 1 KB per page
            migration_time = data_size / (self.dram_bandwidth * 1024 / 1000)  # Time in ms
            remap_time = 0.05  # Fixed remapping time in ms
            self.total_migration_time += migration_time
            self.total_remap_time += remap_time
            self.migration_count += 1

            # Perform migration
            self.dram.write(address, value)
            del self.nvm.data[address]
            print(f"Migrated address {address} to DRAM (Migration Time: {migration_time:.2f} ms, Remap Time: {remap_time:.2f} ms).")
        except MemoryError:
            print("DRAM is full. Cannot migrate.")

    def copy_to_dram(self, address, value):
        # Simulate copying data to DRAM without migration
        data_size = 1024  # Assume 1 KB per page
        copy_time = data_size / (self.dram_bandwidth * 1024 / 1000)  # Time in ms
        self.total_copy_time += copy_time
        self.copy_count += 1
        
        # Perform copy
        self.dram.write(address, value)
        print(f"Copied address {address} to DRAM (Copy Time: {copy_time:.2f} ms).")

    def access_memory(self, address, value=None, write=False, copy=False):
        # Access DRAM first
        data, latency = self.dram.access(address)
        if data is not None:
            print(f"Address {address} found in DRAM. Latency: {latency} ms")
            return latency

        # Access NVM if not found in DRAM
        data, latency = self.nvm.access(address)
        if data is not None:
            print(f"Address {address} found in NVM. Latency: {latency} ms")
            if write:
                if copy:
                    self.copy_to_dram(address, data)
                else:
                    self.migrate_to_dram(address, data)
            return latency

        # If not found, write to NVM
        if write:
            print(f"Address {address} not found. Writing to NVM.")
            self.nvm.write(address, value)
        return latency

    def get_performance_metrics(self):
        return {
            "Total Copy Time (ms)": self.total_copy_time,
            "Total Migration Time (ms)": self.total_migration_time,
            "Total Remap Time (ms)": self.total_remap_time,
            "Total Migrations": self.migration_count,
            "Total Copies": self.copy_count,
        }



In [6]:
# Main function for testing
if __name__ == "__main__":
    tiered_memory = TieredMemorySystem(
        dram_size=10, 
        dram_latency=0.1, 
        dram_bandwidth=2400,  
        nvm_size=100, 
        nvm_latency=0.5, 
        nvm_bandwidth=128  
    )

    hot_addresses = [random.randint(0, 19) for _ in range(20)]
    for _ in range(500):  
        if random.random() < 0.7:
            addr = random.choice(hot_addresses)  
        else:
            addr = random.randint(0, 99)
        # Ensure memory copying is performed
        tiered_memory.access_memory(addr, value=random.randint(0, 100), write=True, copy=True)
        tiered_memory.access_memory(addr)

    metrics = tiered_memory.get_performance_metrics()
    print("Performance Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.2f}")


Address 61 not found. Writing to NVM.
Address 61 found in NVM. Latency: 0.5 ms
Address 2 not found. Writing to NVM.
Address 2 found in NVM. Latency: 0.5 ms
Address 16 not found. Writing to NVM.
Address 16 found in NVM. Latency: 0.5 ms
Address 13 not found. Writing to NVM.
Address 13 found in NVM. Latency: 0.5 ms
Address 91 not found. Writing to NVM.
Address 91 found in NVM. Latency: 0.5 ms
Address 6 not found. Writing to NVM.
Address 6 found in NVM. Latency: 0.5 ms
Address 77 not found. Writing to NVM.
Address 77 found in NVM. Latency: 0.5 ms
Address 11 not found. Writing to NVM.
Address 11 found in NVM. Latency: 0.5 ms
Address 11 found in NVM. Latency: 0.5 ms
Copied address 11 to DRAM (Copy Time: 0.42 ms).
Address 11 found in DRAM. Latency: 0.1 ms
Address 93 not found. Writing to NVM.
Address 93 found in NVM. Latency: 0.5 ms
Address 19 not found. Writing to NVM.
Address 19 found in NVM. Latency: 0.5 ms
Address 93 found in NVM. Latency: 0.5 ms
Copied address 93 to DRAM (Copy Time: 0.42